# 05. 선형대수와 Numpy 프로그래밍 (3)
> 머신러닝에 꼭 필요한 선형대수학 개념들을 배우고 이를 Numpy로 구현하는 방법을 배워봅시다.

- toc: true 
- badges: true
- comments: true
- categories: [Day 1]
- permalink: /linear_algebra_with_numpy_3
- exec: colab

이제 여러분은 선형대수의 프레임워크로 생각할 수 있게 되었습니다. 이제 행렬보다 상위 차원 존재인 텐서에 대해 배우고, 데이터를 자르고 붙이고 돌리는 방법들에 대해 배워봅시다. 이러한 함수들은 실제로 꽤나 유용하며, 이런 함수들을 잘 모르면 산술 프로그래밍 자체가 불가능하니 잘 봐주셔야합니다.

### 1. Tensor

행렬은 기본적으로 벡터가 여러개 쌓여서 만들어진 구조입니다. (기저벡터들이 변환되고 난 뒤의 위치를 가로로 쌓아놓은 구조) 따라서 (n) 혹은 (n, 1)의 Shape를 갖는 벡터들이 m개 쌓이면 Shape이 (n, m)인 행렬이 되는 방식이였습니다.

![img](https://github.com/gusdnd852/bigdata-lecture/blob/master/_notebooks/img/Day1/70.png?raw=true)

그러나 이러한 행렬이 쌓여서 3차원 큐브와 같은 자료를 만들면 어떻게 될까요? 또 그 3차원 큐브가 쌓여서 4차원 데이터를 만들어내면 어떻게 될까요? 이 때 데이터 자체가 몇개의 방향(차원)으로 존재하는지를 우리는 "랭크(Rank)"라고 부릅니다. <br><br>

![img](https://github.com/gusdnd852/bigdata-lecture/blob/master/_notebooks/img/Day1/69.png?raw=true)

스칼라는 랭크가 0, 벡터는 랭크가 1, 행렬은 랭크가 2입니다. 그리고 랭크가 3이상인 모든 데이터들을 "텐서"라고 부르며, 우리는 앞으로 이 텐서를 다룰 것입니다.

### 2. Tensor의 Shape 규칙
Tensor의 경우 Rank가 높기 때문에 Shape을 맞추기 까다롭습니다. 아래와 같은 규칙이 존재합니다.
<br>

![](https://2.bp.blogspot.com/-iuS-Uayk2FA/T_VUQ3nvaiI/AAAAAAAAAhs/ARvCwQFufsc/s1600/matrix_multi.png)
<br><br>

#### 2.1. 3차원 텐서 Shape 규칙
(x, i, j) @ (x, j, k)을 연산할 때 x는 같거나 양쪽 중 한쪽이 1이여야 하며, 맨 끝 두개의 Shape은 (i, j) @ (j, k)에서 j는 동일해야하고, 결과 Shape은 (x, i, k)가 된다.
<br>

- (4, 1, 2) @ (4, 2, 5) = (4, 1, 5)
- (4, 1, 2) @ (3, 2, 5) = 가장 앞자리가 달라서 에러
- (1, 1, 2) @ (3, 2, 5) = (3, 1, 5) 
<br><br>

#### 2.2. 4차원 텐서 Shape 규칙
(x, y, i, j) @ (x, y, j, k)을 연산할 때 x, y는 같거나 양쪽 중 한쪽이 1이여야 하며, 맨 끝 두개의 Shape은 (i, j) @ (j, k)에서 j는 동일해야하고, 결과 Shape은 (x, y, i, k)가 된다.
<br>

- (4, 3, 1, 2) @ (4, 3, 2, 5) = (4, 3, 1, 5)
- (4, 5, 1, 2) @ (3, 5, 2, 5) = 가장 앞자리가 달라서 에러
- (4, 5, 1, 2) @ (4, 3, 2, 5) = 두번째 자리가 달라서 에러
- (4, 5, 1, 2) @ (4, 1, 2, 5) = (4, 5, 1, 5)
- (4, 5, 1, 2) @ (1, 5, 2, 5) = (4, 5, 1, 5)
<br><br>

#### 2.3. n차원 텐서 Shape 규칙
위를 토대로 아래와 같은 일반화가 가능합니다.

(x, ..., z, i, j) @ (x, ..., z, j, k)을 연산할 때 x, ..., z는 같거나 양쪽 중 한쪽이 1이여야 하며, 맨 끝 두개의 Shape은 (i, j) @ (j, k)에서 j는 동일해야하고, 결과 Shape은 (x, ..., z, i, k)가 된다.
<br>


In [23]:
from numpy.random import rand

print((rand(4, 1, 2) @ rand(4, 2, 5)).shape)
print((rand(1, 1, 2) @ rand(3, 2, 5)).shape)
print((rand(4, 3, 1, 2) @ rand(4, 3, 2, 5)).shape)
print((rand(4, 5, 1, 2) @ rand(4, 1, 2, 5)).shape)
print((rand(4, 5, 1, 2) @ rand(1, 5, 2, 5)).shape)

(4, 1, 5)
(3, 1, 5)
(4, 3, 1, 5)
(4, 5, 1, 5)
(4, 5, 1, 5)


<br>

### 3. Axis

Axis는 축이라는 뜻입니다. Shape에서 몇번째 Shape을 지정할지 나타낼 때 많이 사용합니다. 가령 (4, 1, 3)의 Shape을 가진 텐서가 있다면, Axis 0은 4이며, Axis 1은 1, Axis 2는 3이 됩니다. 몇번째 자리수를 나타내는지 사용할 때 씁니다. <br><br>

### 4. 텐서 및 행렬 연산
텐서나 행렬은 매우 다양한 연산을 지원합니다. 아래의 구체적인 예시를 봅시다.

<br>

#### 4.1. Transpose (전치행렬)
$\begin{bmatrix} 1 & 2 & 3 \\ 4 & 5 & 6 \end{bmatrix}^T$ = $\begin{bmatrix} 1 & 4 \\ 2 & 5 \\ 3 & 6 \end{bmatrix}$

트랜스포즈 연산은 행렬을 뒤집는 것을 의미합니다. 만약 Shape가 (2, 3)인 행렬이였다면 뒤집고나서는 Shape가 (3, 2)가 됩니다. 수식 표기는 $A^T$와 같이 T를 붙여서 표기합니다. numpy에서는 .T로 손쉽게 행렬을 트랜스포즈 할 수 있습니다. 

In [28]:
import numpy as np

mat_a = np.array([[1, 2, 3], 
                  [4, 5, 6]])

mat_a.T

array([[1, 4],
       [2, 5],
       [3, 6]])

<br><br>

#### 4.2. Permute : 고 차원 Transpose

텐서의 경우, 랭크가 2보다 높을 수 있습니다. 각 자리의 순서를 바꾸고 싶을 때 Permute 연산을 사용할 수 있습니다. 랭크가 3일 때 기본값은 (0, 1, 2)으로 생각하고, 0번째 1번째 2번째 Shape의 위치를 마음대로 변경합니다. 예를 들어 numpy에서는 transpose(0, 2, 1)이나 transpose(1, 0, 2)처럼 변경 가능합니다.
<br><br>

- Shape : (4, 5, 7) ← ([0]:4, [1]:5, [2]:7)
- transpose(0, 1, 2) : (4, 5, 7)
- transpose(0, 2, 1) : (4, 7, 5)
- transpose(1, 2, 0) : (7, 5, 4)
- transpose(1, 0, 2) : (5, 4, 7)

In [42]:
mat_a = np.array([[[1, 2, 3], [4, 5, 6]],
                  [[1, 2, 3], [4, 5, 6]],
                  [[1, 2, 3], [4, 5, 6]],
                  [[1, 2, 3], [4, 5, 6]]])


print('Shape : ', mat_a.shape)
print('transpose(0, 1, 2) : ', mat_a.transpose(0, 1, 2).shape)
print('transpose(0, 2, 1) : ', mat_a.transpose(0, 2, 1).shape)
print('transpose(1, 0, 2) : ', mat_a.transpose(1, 0, 2).shape)
print('transpose(1, 2, 0) : ', mat_a.transpose(1, 2, 0).shape)

Shape :  (4, 2, 3)
transpose(0, 1, 2) :  (4, 2, 3)
transpose(0, 2, 1) :  (4, 3, 2)
transpose(1, 0, 2) :  (2, 4, 3)
transpose(1, 2, 0) :  (2, 3, 4)


<br><br>

### 4.3. Reshpae
![](https://i.stack.imgur.com/OUjlv.png)
<br>

Reshape 연산은 행렬이나 텐서의 일부분을 떼내서 다른 모양으로 재조직합니다. 때문에 Shape을 마음대로 바꿀 수 있습니다. 단 일정한 규칙 안에서 변경 가능한데, 해당하는 텐서의 Shape에서 각 Axis들의 곱으로 만들어낼 수 있는 Shape만 변경 가능합니다. numpy에서는 reshape()이라는 함수로 사용 가능합니다. 아래 예시를 봅시다.
<br><br>

- (4, 2, 3) → reshape(4, 2 * 3) → (4, 6)
- (4, 3, 2) → reshape(4 * 2, 3) → (8, 3)
- (4, 3, 2) → reshape(4 * 2 * 3, 1) → (24, 1)
- (4, 3, 2) → reshape(1, 4 * 2 * 3) → (1, 24)

In [47]:
mat_a = np.array([[[1, 2, 3], [4, 5, 6]],
                  [[1, 2, 3], [4, 5, 6]],
                  [[1, 2, 3], [4, 5, 6]],
                  [[1, 2, 3], [4, 5, 6]]])


print('Shape : ', mat_a.shape)
print('reshape(4, 2 * 3) : ', mat_a.reshape(4, 2 * 3).shape)
print('reshape(4 * 2, 3) : ', mat_a.reshape(4 * 2, 3).shape)
print('reshape(4 * 2 * 3) : ', mat_a.reshape(4 * 2 * 3).shape)
print('reshape(1, 4 * 2 * 3) : ', mat_a.reshape(1, 4 * 2 * 3).shape)
print('reshape(4 * 2 * 3, 1) : ', mat_a.reshape(4 * 2 * 3, 1).shape)

Shape :  (4, 2, 3)
reshape(4, 2 * 3) :  (4, 6)
reshape(4 * 2, 3) :  (8, 3)
reshape(4 * 2 * 3) :  (24,)
reshape(1, 4 * 2 * 3) :  (1, 24)
reshape(4 * 2 * 3, 1) :  (24, 1)


<br>

#### 4.4. Squeeze
Squeeze연산은 크기가 1인 Axis를 제거합니다. 텐서를 쥐어짜서 필요 없는 차원을 없앤다고 봐도 무방합니다 (그래서 이름이 squeeze인듯 합니다.)

- Shape : (4, 1, 2) → squeeze : (4, 2)
- Shape : (4, 1, 1, 1, 2) → squeeze : (4, 2)
- Shape : (1, 4, 1, 2) → squeeze : (4, 2)

In [53]:
from numpy.random import rand


print('origin : ', rand(4, 1, 2).shape, 'squeeze : ', rand(4, 1, 2).squeeze().shape)
print('origin : ', rand(4, 1, 1, 1, 2).shape, 'squeeze : ', rand(4, 1, 1, 1, 2).squeeze().shape)
print('origin : ', rand(1, 4, 1, 2).shape, 'squeeze : ', rand(1, 4, 1, 2).squeeze().shape)

origin :  (4, 1, 2) squeeze :  (4, 2)
origin :  (4, 1, 1, 1, 2) squeeze :  (4, 2)
origin :  (1, 4, 1, 2) squeeze :  (4, 2)


<br>

#### 4.4. Expand_dims (== unsqueeze)
Expand_dims연산은 Unsqueeze라고도 부르는데 Squeeze와 반대로 원하는 Axis에 크기가 1인 Axis를 새로 만듭니다.

- Shape : (4, 2) → expand_dims(0) : (1, 4, 2)
- Shape : (4, 2) → expand_dims(1) : (4, 1, 2)
- Shape : (4, 2) → expand_dims(2) : (4, 2, 1)

In [54]:
from numpy.random import rand


print('expand_dims(0) : ', rand(4, 2).expand_dims(0).shape)
print('expand_dims(1) : ', rand(4, 2).expand_dims(1).shape)
print('expand_dims(2) : ', rand(4, 2).expand_dims(2).shape)

AttributeError: 'numpy.ndarray' object has no attribute 'expand_dims'